### Instructions
This script is for training a binary classification model CNN from the given dataset using Keras + Tensorflow stack.
1. Configure the paramers as required
2. Configure the model as required
3. Run all the cells

### Imports

imports the required modules from keras, Tensorflow, PIL, numpy and custom file handling functions

In [1]:
from urllib.request import urlretrieve
from urllib.parse import urlencode
import image_slicer
import traceback
from os import listdir, remove, mkdir
from os.path import join, exists, expanduser, isdir
from shutil import rmtree
from utils.file_utils import make_dir_helper, is_jpeg, delete_dir_helper
from PIL import Image
from keras.preprocessing import image
from keras import optimizers, regularizers, models, layers
from keras.models import load_model, model_from_json
import numpy as np

ModuleNotFoundError: No module named 'image_slicer'

### Configurable parameters

Defines various constants, directory paths and design of the model

In [ ]:
URL = 'http://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/export?'
x_min = -117.261067
y_min = 33.052972
x_max = -117.256993
y_max = 33.054659
extent = {'xmin': -13043537.000969624, 'ymin': 4035872.34275381, 'xmax': -13042340.283745637, 'ymax': 4036350.07418055}
spatial_ref = 4326
width = 1000
height = 1000

In [20]:
def crop(map_file_path, sliced_dir, height, width):
    
    with Image.open(map_file_path) as im:
        imgwidth, imgheight = im.size
        idx = 0
        for i in range(0,imgheight - height , height): # Don't include borders
            for j in range(0,imgwidth - width , width):
                box = (j, i, j+width, i+height)
                try:
                    a = im.crop(box)
                    a.save(join(sliced_dir, 'image_{}.jpg'.format(idx)), "JPEG", quality=100, optimize=True)
                except:
                    traceback.print_exc()
                    pass
                idx +=1

In [21]:
def get_class_names(base_dir):
    return [dname for dname in listdir(base_dir) if isdir(join(base_dir, dname))]

In [22]:
def predict_class(model, data_dir, pred_dir, class_names):
    
    image_size = 256
    
    model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])
    model.summary()
    
    fnames = [fname for fname in listdir(data_dir) if is_jpeg(fname)]
    print(fnames)
    
    for idx, fname in enumerate(fnames):
        
        img_path = join(data_dir, fname)
        
        img = image.load_img(img_path, target_size=(IMAGE_SIZE, IMAGE_SIZE)) 
        x = image.img_to_array(img)
        x = x.reshape((1,) + x.shape)
        
        prediction = model.predict(x)[0]
        max_index = np.argmax(prediction)
        max_val = np.amax(prediction)
        
        pred_class = class_names[max_index]
        pred_class_dir = join(pred_dir, pred_class)
        make_dir_helper(pred_class_dir)
        
        img.save(join(pred_class_dir, 'image_{}.jpg'.format(idx)), "JPEG", quality=100, optimize=True)
        

In [23]:
parameters = {
    'bbox':'{},{},{},{}'.format(x_min, y_min, x_max, y_max),
    'bboxSR':spatial_ref,
    'size':'{},{}'.format(width, height),
    'format':'jpg',
    'f':'image'
}

URL += urlencode(parameters)

PROJECT_DIR = join(expanduser('~'),'desktop','Esri Project');
MAP_NAME = 'map.jpg'
MAP_DIR = join(PROJECT_DIR, 'maps')
SLICED_DIR = join(MAP_DIR, 'sliced')
PRED_DIR = join(MAP_DIR, 'predicted')
MODEL_FILE_PATH = join(PROJECT_DIR, 'models', 'map_multi_model_7.h5')
IMAGE_SIZE = 150


CLASSES = get_class_names(join(PROJECT_DIR, 'original data 2'))

delete_dir_helper(SLICED_DIR)

make_dir_helper(MAP_DIR)
make_dir_helper(SLICED_DIR)
make_dir_helper(PRED_DIR)


MAP_FILE_PATH = join(MAP_DIR, MAP_NAME)


if urlretrieve(URL, MAP_FILE_PATH):

    tile_size = 200
    crop(MAP_FILE_PATH, SLICED_DIR, IMAGE_SIZE, IMAGE_SIZE)
    model = load_model(MODEL_FILE_PATH)
    predict_class(model, SLICED_DIR, PRED_DIR, CLASSES)
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_5 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 9)                 2313      
Total params: 16,814,409
Trainable params: 2,099,721
Non-trainable params: 14,714,688
_________________________________________________________________
['image_0.jpg', 'image_1.jpg', 'image_10.jpg', 'image_11.jpg', 'image_12.jpg', 'image_13.jpg', 'image_14.jpg', 'image_15.j